# Transfer Learning with Tensorflow Part 3: Scaling up

We've seen the power of transfer learning featuter extraction and fine-tuning, now its time to scale up to all of the classes in Food101. (101 total classes of food)

Our goal is to beat the original Food101 paper.

Baseline: 50.67%

In [33]:
import tensorflow as tf
gpu = tf.config.list_physical_devices('GPU')
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Creating helper functions

In previous notebooks we've created series of helper functions but it is difficult to do that every single time so let's download them

In [34]:
def download_helper_functions():
    !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [35]:
#Download the helper function
import os
path = os.getcwd()
if not os.path.exists('/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Transfer Learning/helper_functions.py'):
    download_helper_functions()

In [36]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## 101 Food Classes: working with less data

Our goal is to beat the original Food101 paper with 10% of the training data

The data we're downlaoding comes from the original Food101 but was preprocessed

In [37]:
if not os.path.exists('/Users/klsharma22/Desktop/TensorflowCertifcationCourse/Transfer Learning/101_food_classes_10_percent.zip'):
    !wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
    unzip_data('101_food_classes_10_percent.zip')

In [38]:
TRAIN_DIR = '101_food_classes_10_percent/train/'
TEST_DIR = '101_food_classes_10_percent/test/'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
LABEL_MODE = 'categorical'

In [39]:
# How many images classes are there
walk_through_dir('101_food_classes_10_percent')

There are 2 directories and 0 images in '101_food_classes_10_percent'.
There are 101 directories and 0 images in '101_food_classes_10_percent/test'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/foie_gras'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/club_sandwich'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/cheese_plate'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/cup_cakes'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/garlic_bread'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/gnocchi'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/samosa'.
There are 0 directories and 250 images in '101_food_classes_10_percent/test/donuts'.
There are 0 directories and 250 images in '101_food_class

In [40]:
# Setup data inputs
import tensorflow as tf

train_data = tf.keras.preprocessing.image_dataset_from_directory(TRAIN_DIR, label_mode= LABEL_MODE, image_size= IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(TEST_DIR, label_mode= LABEL_MODE, image_size= IMG_SIZE, shuffle= False)

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Train a big dog model with transfer learning on 10% of 101 food classes

Steps we're going to take:
* Create a ModelCcheckpoint callback
* Create a data augmnetation layer to build data augmentation right into the model
* Build a headless Functional EfficientNetB0 backbone-model
* Comple our model
* Feature extract for 5 full passes (15% of test dataset which will be the validation dataset   )

In [41]:
# Create checkpoint callback
checkpoint_path = '101_classes_10_percent_data_model_checkpoint'
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only= True, monitor= "val_accuracy", save_best_only= True)

In [42]:
# Create data augmentation layer to incorporate it right into the model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomHeight(0.2),
    layers.RandomWidth(0.2),
    layers.RandomZoom(0.2),
    # preprocessing.Rescalinig(1/255.) required onlu if the data scaling is not present in the model
], name= 'data_augmentation_layer')

In [43]:
# Setup the base model
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top= False)
base_model.trainable = False

# Setup model archtecture with trainable top layers
inputs = layers.Input(shape= IMG_SIZE + (3, ), name= "input_layer")
x = data_augmentation(inputs)
x = base_model(x, training= False) # put the base model n inference mode so the weights which need to stay frozen stay as it is
x = layers.GlobalAveragePooling2D(name= "global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data.class_names), activation= "softmax", name= "output_layer")(x)

model = tf.keras.Model(inputs, outputs)


In [44]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layer (S  (None, None, None, 3)     0         
 equential)                                                      
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_avg_pool_layer (Glo  (None, 1280)              0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                           

In [45]:
# Compiilng the model
model.compile(loss= 'categorical_crossentropy',
              optimizer= tf.keras.optimizers.legacy.Adam(),
              metrics= ['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
    history_10_percent = model.fit(train_data,
                                epochs= 5,
                                steps_per_epoch= len(train_data),
                                validation_data= test_data,
                                validation_steps= int(0.15 * len(test_data)))

Epoch 1/5


2024-02-21 21:33:52.010184: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


  1/237 [..............................] - ETA: 32:11 - loss: 4.7078 - accuracy: 0.0000e+00

2024-02-21 21:33:58.425665: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


  2/237 [..............................] - ETA: 18:19 - loss: 4.6577 - accuracy: 0.0000e+00

...

 88/237 [==========>...................] - ETA: 7:39 - loss: 1.8745 - accuracy: 0.5575

2024-02-21 22:02:07.586994: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 89/237 [==========>...................] - ETA: 7:37 - loss: 1.8760 - accuracy: 0.5572

2024-02-21 22:02:11.159475: I metal_plugin/src/kernels/stateless_random_op.cc:282] Note the GPU implementation does not produce the same series as CPU implementation.


 90/237 [==========>...................] - ETA: 7:32 - loss: 1.8767 - accuracy: 0.5569

In [ ]:
model.evaluate(test_data)

In [ ]:
plot_loss_curves(history_10_percent)

> Accordinig to the history, the model is not learning in the correct manner so we need to fine-tune the model. The model is over-fitting

## Fine-tuning

In [ ]:
# Unfreeze all of the layers iin the base model
base_model.trainable = True

# Refreeze every layer except the last 5
for layer in base_model.layers[:-5]:
    layer.trainable= False

In [ ]:
# Recompile the model using lower learning rate
model.compile(loss= 'categorical_crossentropy',
              optimizer= tf.keras.optimizers.legacy.Adam(learning_rate= 0.0001),
              metrics= ['accuracy'])

In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable)

In [ ]:
for i, layer in enumerate(model.layers[2].layers):
    print(i, layer.name, layer.trainable)

In [ ]:
fine_tune_epochs = 10

In [ ]:
tf.config.set_physical_device_memory_limit('GPU:0', desired_memory_limit)

In [ ]:
tf.random.set_seed(42)
tf.config.set_soft_device_placement(True)
with tf.device('/device:GPU:0'):
    history_10_percent_fine_tune = model.fit(train_data,
            epochs= fine_tune_epochs,
            validation_data= test_data,
            validation_steps= int(0.15 * len(test_data)),
            initial_epoch= 5)

In [ ]:
model.evaluate(test_data)

In [ ]:
compare_historys(original_history= history_10_percent,
                 new_history= history_10_percent_fine_tune)